> Notes: [Web Scraping](https://automatetheboringstuff.com/chapter11/)

In [6]:
# modules that make it easy to scrape web pages in Python.
# (1) webbrowser: comes with Python and opens a browser to a specific page.
# (2) Requests: downloads files and web pages from the Internet.
#     Requests is easier and faster to use then urllib2.
# (3) Beautiful Soup: parses HTML, the format that web pages are written in.
# (4) Selenium: launches and controls a web browser. 
#     Selenium is able to fill in forms and simulate mouse clicks in this browser.
# (5) pyperclip: A cross-platform clipboard module for Python. (only handles plain text for now)

In [7]:
import webbrowser

In [8]:
webbrowser.open('http://inventwithpython.com/')

True

In [10]:
import webbrowser, sys, pyperclip 

In [54]:
def map_it(*args, **kwargs):
    if (len(args)>0):
        address = ''.join(args[:])
        print('args: address: {}'.format(address))
    elif (len(kwargs)>0):
        values = []
        for k,v in kwargs.items():
            values.append(v)
        address = ''.join(values)
        print('kwargs: address: {}'.format(address))
    #elif (len(sys.argv)>1):
        #address = ' '.join(sys.argv[1:])
        #print('argv: address: {}'.format(address))
    else:
        address = pyperclip.paste()
        print('pyperclip: address: {}'.format(address))
    webbrowser.open('https://www.google.com/maps/place/' + address)

#map_it('870 Valencia St,',' San Francisco,',' CA 94110')    
#map_it(add1='870 Valencia St,',add2=' San Francisco,',add3=' CA 94110')    
#map_it()


pyperclip: address: '870 Valencia St, San Francisco, CA 94110'


In [81]:
from lxml import html
import requests, webbrowser, time

In [95]:
def open_tab(url, sleep_time):
    print('sleeping for {} before opening {}'.format(sleep_time, url))
    time.sleep(sleep_time)
    webbrowser.open_new_tab(url)
    
def open_page_links_in_browser_tabs(url, max_page_urls_to_open, sleep_time):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    hrefs = tree.xpath('//a/@href')
    hrefs_non_relative = []
    for href in hrefs:
        if 'http' in href:
            hrefs_non_relative.append(href)
    for href in hrefs_non_relative[:max_page_urls_to_open]:
        open_tab(href,sleep_time)

In [96]:
open_page_links_in_browser_tabs('https://automatetheboringstuff.com/',5,2)

sleeping for 2 before opening https://automatetheboringstuff.com/
sleeping for 2 before opening https://www.nostarch.com/automatestuff
sleeping for 2 before opening http://www.amazon.com/gp/product/1593275994/ref=as_li_tl?ie=UTF8&camp=1789&creative=9325&creativeASIN=1593275994&linkCode=as2&tag=playwithpyth-20&linkId=HDM7V3T6RHC5VVN4
sleeping for 2 before opening https://twitter.com/AlSweigart
sleeping for 2 before opening https://creativecommons.org/licenses/by-nc-sa/3.0/


In [169]:
# parsing json - http://docs.python-guide.org/en/latest/scenarios/json/
import requests, json, webbrowser
#res = requests.get('https://ipinfo.io/')
res = requests.get('http://www.google.com/search?q=local zip code')
tree = html.fromstring(res.content)
divs = tree.xpath('//b/text()')
print(divs)
#print(type(res))
#print(res.status_code == requests.codes.ok)
#print(len(res.text))
#print(res.text)
#txt = json.loads(res.text[:])
#local_zip = txt['postal']
#print(local_zip)
#webbrowser.open('https://weather.com/weather/today/l/'+local_zip)

['Search', 'Zip Code', 'zip', 'Zip Codes', 'Zip Code', 'Zip Code', 'zip', 'Zip Codes', 'Zip Code', 'zip', 'Zip Codes', 'zip code', 'zip', 'ZIP code', 'ZIP Code', 'zipcodes', '06902', '06902', 'ZIP code', 'zip code', 'zipcodes', 'Zip Codes', 'zip code', 'zip codes', 'local', 'Zip Code', 'zip', 'Zip Code', 'Zip Codes', 'Zip Code', 'zip', 'Zip code', 'zip code', 'Zip code', 'Zip code', 'Zip Code', 'zip', 'Zip Code', 'zip code', 'local', 'Zip Codes', 'ZipCode', 'zipcode', 'Zip Codes', 'zip codes', 'local zip code', 'stamford', 'map', 'stamford', '06902', 'downtown stamford', 'stamford ny', 'stamford ct', 'area', 'stamford tx', 'stamford ct area', 'stamford ct 9 digit', '1']
